In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/ai agent trainning/train/sft')

import sys
import os
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

Mounted at /content/drive


In [ ]:
!pip install -q bitsandbytes accelerate peft trl==0.15.2 datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import yaml

with open('../configs/base.yaml', 'r') as file:
    configs = yaml.safe_load(file)

with open('../configs/ppo.yaml', 'r') as file:
    configs.update(yaml.safe_load(file))

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model

In [ ]:
from transformers import AutoTokenizer
from peft import AutoPeftModelForCausalLM, AutoPeftModelForSequenceClassification

# sft
sft_model = AutoPeftModelForCausalLM.from_pretrained(
    configs['sft_model'],
    load_in_4bit = configs['model']['load_in_4bit'],
    torch_dtype=torch.bfloat16,
)
# ref
ref_model = AutoPeftModelForCausalLM.from_pretrained(
    configs['sft_model'],
    load_in_4bit = configs['model']['load_in_4bit'],
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(configs['sft_model'])

# llm backbone
backbone = AutoPeftModelForCausalLM.from_pretrained(
    configs['llm_backbone'],
    load_in_4bit = configs['model']['load_in_4bit'],
    torch_dtype=torch.bfloat16,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:195: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [ ]:
from reward import RewardModel
from peft import PeftModel

reward_model = RewardModel(backbone, True)
reward_model = PeftModel.from_pretrained(reward_model, configs['reward_model'])
reward_model.reward_head = torch.load(configs['reward_model']+"/reward_head.pt")

/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['corda_config', 'trainable_token_indices'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(
<ipython-input-6-f6186dd5403c>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed du

In [ ]:
from value import ValueModel

value_model = ValueModel(sft_model)

# Load data

In [ ]:
import json

with open("../../data/RL/health-care-prompt.json", "r") as f:
    data = json.load(f)
data = data[:10]
len(data)

10

In [ ]:
test_size = int(0.2 * len(data))

In [ ]:
from datasets import Dataset

dataset = Dataset.from_list([{'prompt': d} for d in data[test_size:]])
eval_dataset = Dataset.from_list([{'prompt': d} for d in data[test_size:]])

In [ ]:
def custom_collate_fn(batch):
    input_ids = [tokenizer(item['prompt']) for item in batch]
    input_ids = [torch.tensor(item['input_ids']) for item in input_ids]

    # Padding để các tensor có cùng chiều dài
    max_len = max(len(ids) for ids in input_ids)
    padded_input_ids = []

    for ids in input_ids:
        padding = torch.tensor([tokenizer.pad_token_id] * (max_len - len(ids)))
        padded = torch.cat([ids, padding])
        padded_input_ids.append(padded)

    # Stack các tensor đã được padding thành một tensor duy nhất
    input_ids_tensor = torch.stack(padded_input_ids).to(torch.int)

    return {'input_ids': input_ids_tensor}

# Config model

In [ ]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(
    # task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=configs['model']['r'],
    lora_alpha=configs['model']['lora_alpha'],
    lora_dropout=configs['model']['lora_dropout'],
    target_modules =configs['model']['target_modules'],
    bias = configs['model']['bias'],

    use_rslora = configs['model']['use_rslora'],
    # loftq_config = configs['model']['loftq_config'],
)

In [ ]:
from trl import PPOTrainer, PPOConfig

is_bf_16 = True

# Set up PPO configuration
ppo_config = PPOConfig(
    per_device_train_batch_size=configs['ppo']['per_device_train_batch_size'],
    gradient_accumulation_steps=configs['ppo']['gradient_accumulation_steps'],
    # bf16= is_bf_16, # not fb16
    # fp16= not is_bf_16, # not bf16
    logging_steps=configs['trainning']['logging_steps'],

    # num_train_epochs=configs['trainning']['num_train_epochs'],
    max_steps = configs['trainning']['max_steps'],

    learning_rate=configs['trainning']['learning_rate'],
    weight_decay=configs['trainning']['weight_decay'],
    report_to=configs['trainning']['report_to'],
    seed = configs['trainning']['seed'],
    warmup_steps = configs['trainning']['warmup_steps'],
    optim = configs['trainning']['optim'],
    lr_scheduler_type = configs['trainning']['lr_scheduler_type'],
    output_dir = configs['trainning']['output_dir'],

    # PPO train
    num_ppo_epochs=configs['ppo']['num_ppo_epochs'],
    kl_coef=configs['ppo']['kl_coef'],
    # kl_estimator=configs['ppo']['kl_estimator'],
    cliprange=configs['ppo']['cliprange'],
    vf_coef=configs['ppo']['vf_coef'],
    cliprange_value=configs['ppo']['cliprange_value'],
    gamma=configs['ppo']['gamma'],
    lam=configs['ppo']['lam'],

    # eval_strategy='no'
)

# Train

In [ ]:
# Initialize PPOTrainer
ppo_trainer = PPOTrainer(
    model           = sft_model,
    ref_model       = ref_model,
    value_model     = value_model,
    reward_model    = reward_model,

    processing_class=tokenizer,
    data_collator=custom_collate_fn,
    train_dataset=dataset,
    eval_dataset=eval_dataset,

    args=ppo_config,
    peft_config=peft_config,
)

for param in ppo_trainer.model.value_model.value_head.parameters():
    param.requires_grad = True

ppo_trainer.train()

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


===training policy===


From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


Step,Training Loss


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ query                                         ┃ model response                                 ┃ score          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ Xin chào, tôi đang gặp vấn đề với một khối u  │  (bạch huyết là một loại ung thư)              │ 0.08447265625  │
│ sưng tấy ở một bên cổ gần cằm. đôi khi nó sẽ  │ Không, khối u sưng tấy ở cổ không phải là sưng │                │
│ rất đau và khối u sẽ “bùng nổ” đột ngột và    │ hạch bạch huyết. Bạch huyết là một loại ung    │                │
│ tôi có thể cảm thấy vị đắng đang trào lên cổ  │ thư, và nó thường xuất hiện ở các vị trí khác  │                │
│ họng mình. đó có phải là sưng hạch bạch huyết │ nhau trên cơ                                   │                │
│ không?                                        │                                                │                │
├───────────────────────────────────────────────┼────────────────────────────────────────────────┼────────────────┤
│ Xin chào, không ngờ lại gặp được người thật,  │  (có thể có một số vấn đề về tim, nhưng tôi    │ -0.1435546875  │
│ nhưng vì bạn ở đây nên tôi sẽ kể cho bạn nghe │ không chắc chắn về điều đó). Tôi rất lo lắng   │                │
│ về những triệu chứng mà tôi đang gặp phải.    │ và không biết phải làm gì tiếp theo. Tôi có    │                │
│ Tôi nghĩ đó là những cơn hoảng loạn hoặc lo   │ thể hỏi thêm thông tin về các triệu chứng này  │                │
│ lắng, nhưng ai biết được. Tôi có cảm giác     │ và có thể có một số nguyên                     │                │
│ căng cứng ở phía sau đầu, di chuyển lên thái  │                                                │                │
│ dương rồi xuống cổ họng và cuối cùng xuống    │                                                │                │
│ giữa ngực. Tôi rất xúc động khi điều này xảy  │                                                │                │
│ ra và sau tất cả những điều đó, tôi không thể │                                                │                │
│ thở được và tim tôi đập rất nhanh. Nó thường  │                                                │                │
│ biến mất sau 2 hoặc 3 phút nếu tôi có thể     │                                                │                │
│ ngồi xuống và tập trung vào hơi thở. Gần đây  │                                                │                │
│ tôi đã chụp MRI và tim tôi vẫn ổn......có ý   │                                                │                │
│ kiến ​​gì không????                             │                                                │                │
├───────────────────────────────────────────────┼────────────────────────────────────────────────┼────────────────┤
│ Tôi 36 tuổi, đang mang thai 12 tuần. Tôi có   │  (chị em).                                     │ -0.17578125    │
│ một con gái 11 tuổi. Tôi có tiền sử bệnh PCod │                                                │                │
│ và lạc nội mạc tử cung. Tôi bị sảy thai trong │ Trả lời                                        │                │
│ lần mang thai đầu tiên và đó là sinh ba. cả   │                                                │                │
│ thai kỳ của tôi đều nhờ điều trị nhưng lần    │ Bạn đang mang thai và có một số thắc mắc về    │                │
│ này nhờ ơn Chúa, tôi thụ thai tự nhiên. Tôi   │ điều trị và tình trạng sức khỏe của bạn. Tôi   │                │
│ được tiêm fertygen hai lần một tuần, viên     │ sẽ cố gắng trả lời các thắc mắc của bạn một    │                │
│ nang progestrone và bây giờ là viên nén nhưng │ cách chi tiết nhất có thể.                     │                │
│ hôm qua tôi bị chảy máu với những cục máu     │                                                │                │
│ đông lớn sau khi chụp chiếu, em bé vẫn ổn bên │ Bạn 

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ query                                         ┃ model response                                ┃ score           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Xin chào, tôi đang gặp vấn đề với một khối u  │  (Tin nhắn đã được chỉnh sửa để phản ánh vấn  │ 0.1484375       │
│ sưng tấy ở một bên cổ gần cằm. đôi khi nó sẽ  │ đề được nêu rõ hơn)                           │                 │
│ rất đau và khối u sẽ “bùng nổ” đột ngột và    │                                               │                 │
│ tôi có thể cảm thấy vị đắng đang trào lên cổ  │ Tôi không chắc chắn về tình trạng này và tôi  │                 │
│ họng mình. đó có phải là sưng hạch bạch huyết │ muốn được tư vấn về tình trạng này. Tôi đang  │                 │
│ không?                                        │ lo lắng về sức khỏe của mình và tôi muốn được │                 │
│                                               │ tư                                            │                 │
├───────────────────────────────────────────────┼───────────────────────────────────────────────┼─────────────────┤
│ Xin chào, không ngờ lại gặp được người thật,  │  Tôi rất lo lắng về tình trạng này và tôi     │ -0.140625       │
│ nhưng vì bạn ở đây nên tôi sẽ kể cho bạn nghe │ không biết liệu tôi có đang gặp vấn đề sức    │                 │
│ về những triệu chứng mà tôi đang gặp phải.    │ khỏe nghiêm trọng hay chỉ là vấn đề về tâm    │                 │
│ Tôi nghĩ đó là những cơn hoảng loạn hoặc lo   │ thần. Tôi rất lo lắng và tôi không biết tôi   │                 │
│ lắng, nhưng ai biết được. Tôi có cảm giác     │ có thể được đánh giá hay không. Tôi rất cảm   │                 │
│ căng cứng ở phía sau đầu, di chuyển lên thái  │                                               │                 │
│ dương rồi xuống cổ họng và cuối cùng xuống    │                                               │                 │
│ giữa ngực. Tôi rất xúc động khi điều này xảy  │                                               │                 │
│ ra và sau tất cả những điều đó, tôi không thể │                                               │                 │
│ thở được và tim tôi đập rất nhanh. Nó thường  │                                               │                 │
│ biến mất sau 2 hoặc 3 phút nếu tôi có thể     │                                               │                 │
│ ngồi xuống và tập trung vào hơi thở. Gần đây  │                                               │                 │
│ tôi đã chụp MRI và tim tôi vẫn ổn......có ý   │                                               │                 │
│ kiến ​​gì không????                             │                                               │                 │
├───────────────────────────────────────────────┼───────────────────────────────────────────────┼─────────────────┤
│ Tôi 36 tuổi, đang mang thai 12 tuần. Tôi có   │  - Anh Hùng. Anh Hùng là một bác sĩ. Anh Hùng │ -0.1728515625   │
│ một con gái 11 tuổi. Tôi có tiền sử bệnh PCod │ là một bác sĩ. Anh Hùng là một bác sĩ. Anh    │                 │
│ và lạc nội mạc tử cung. Tôi bị sảy thai trong │ Hùng là một bác sĩ. Anh Hùng là một bác sĩ.   │                 │
│ lần mang thai đầu tiên và đó là sinh ba. cả   │ Anh Hùng là một bác sĩ.                       │                 │
│ thai kỳ của tôi đều nhờ điều trị nhưng lần    │                                               │                 │
│ này nhờ ơn Chúa, tôi thụ thai tự nhiên. Tôi   │                                               │                 │
│ được tiêm fertygen hai lần một tuần, viên     │                                               │                 │
│ nang progestrone và bây giờ là viên nén nhưng │                                               │                 │
│ hôm qua tôi bị chảy máu với những cục máu     │     

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                              ┃ score               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ Xin chào, tôi đang gặp vấn đề với một khối  │  (Tin tức sức khỏe được kiểm tra bởi các    │ 0.1328125           │
│ u sưng tấy ở một bên cổ gần cằm. đôi khi nó │ chuyên gia y tế. Tuy nhiên, tình trạng này  │                     │
│ sẽ rất đau và khối u sẽ “bùng nổ” đột ngột  │ có thể được gây ra bởi các vấn đề khác. Nếu │                     │
│ và tôi có thể cảm thấy vị đắng đang trào    │ bạn lo lắng về tình trạng này, hãy tham     │                     │
│ lên cổ họng mình. đó có phải là sưng hạch   │ khảo ý kiến của bác sĩ.) Tôi đang           │                     │
│ bạch huyết không?                           │                                             │                     │
├─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────┤
│ Xin chào, không ngờ lại gặp được người      │  Tôi rất lo lắng và tôi không biết liệu     │ -0.1533203125       │
│ thật, nhưng vì bạn ở đây nên tôi sẽ kể cho  │ điều này có liên quan đến bệnh tim hoặc     │                     │
│ bạn nghe về những triệu chứng mà tôi đang   │ bệnh tim mạch hoặc có vấn đề khác nào       │                     │
│ gặp phải. Tôi nghĩ đó là những cơn hoảng    │ không. Tôi rất cảm ơn sự trợ giúp của bạn.  │                     │
│ loạn hoặc lo lắng, nhưng ai biết được. Tôi  │ Tôi rất lo lắng về tình trạng sức khỏe của  │                     │
│ có cảm giác căng cứng ở phía sau đầu, di    │ mình. Tôi                                   │                     │
│ chuyển lên thái dương rồi xuống cổ họng và  │                                             │                     │
│ cuối cùng xuống giữa ngực. Tôi rất xúc động │                                             │                     │
│ khi điều này xảy ra và sau tất cả những     │                                             │                     │
│ điều đó, tôi không thể thở được và tim tôi  │                                             │                     │
│ đập rất nhanh. Nó thường biến mất sau 2     │                                             │                     │
│ hoặc 3 phút nếu tôi có thể ngồi xuống và    │                                             │                     │
│ tập trung vào hơi thở. Gần đây tôi đã chụp  │                                             │                     │
│ MRI và tim tôi vẫn ổn......có ý kiến ​​gì     │                                             │                     │
│ không????                                   │                                             │                     │
├─────────────────────────────────────────────┼─────────────────────────────────────────────┼─────────────────────┤
│ Tôi 36 tuổi, đang mang thai 12 tuần. Tôi có │  - Anh Hùng. Anh Hùng là một bác sĩ. Anh    │ -0.1728515625       │
│ một con gái 11 tuổi. Tôi có tiền sử bệnh    │ Hùng là một bác sĩ. Anh Hùng là một bác sĩ. │                     │
│ PCod và lạc nội mạc tử cung. Tôi bị sảy     │ Anh Hùng là một bác sĩ. Anh Hùng là một bác │                     │
│ thai trong lần mang thai đầu tiên và đó là  │ sĩ. Anh Hùng là một bác sĩ.                 │                     │
│ sinh ba. cả thai kỳ của tôi đều nhờ điều    │                                             │                     │
│ trị nhưng lần này nhờ ơn Chúa, tôi thụ thai │                                             │                     │
│ tự nhiên. Tôi được tiêm fertygen hai lần    │                                             │                     │
│ một tuần, viên nang progestrone và bây giờ  │                                             │                     │
│ là viên nén nhưng hôm qua tôi bị chảy máu   │       

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ query                                         ┃ model response                               ┃ score            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Xin chào, tôi đang gặp vấn đề với một khối u  │  Tôi đang lo lắng về tình trạng này và tôi   │ 0.1396484375     │
│ sưng tấy ở một bên cổ gần cằm. đôi khi nó sẽ  │ muốn được tư vấn về tình trạng này. Tôi đang │                  │
│ rất đau và khối u sẽ “bùng nổ” đột ngột và    │ bị bệnh hay tôi chỉ đang bị viêm nhiễm? Tôi  │                  │
│ tôi có thể cảm thấy vị đắng đang trào lên cổ  │ đang gặp vấn đề về sức khỏe nào khác? Tôi    │                  │
│ họng mình. đó có phải là sưng hạch bạch huyết │ cần được đánh giá và tư vấn về tình trạng    │                  │
│ không?                                        │                                              │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ Xin chào, không ngờ lại gặp được người thật,  │  Tôi rất cảm ơn cho sự trợ giúp và sự đánh   │ -0.138671875     │
│ nhưng vì bạn ở đây nên tôi sẽ kể cho bạn nghe │ giá. Tôi rất cảm ơn. Tôi rất cảm ơn. Tôi rất │                  │
│ về những triệu chứng mà tôi đang gặp phải.    │ cảm ơn. Tôi rất cảm ơn. Tôi rất cảm ơn. Tôi  │                  │
│ Tôi nghĩ đó là những cơn hoảng loạn hoặc lo   │ rất cảm ơn. Tôi rất cảm                      │                  │
│ lắng, nhưng ai biết được. Tôi có cảm giác     │                                              │                  │
│ căng cứng ở phía sau đầu, di chuyển lên thái  │                                              │                  │
│ dương rồi xuống cổ họng và cuối cùng xuống    │                                              │                  │
│ giữa ngực. Tôi rất xúc động khi điều này xảy  │                                              │                  │
│ ra và sau tất cả những điều đó, tôi không thể │                                              │                  │
│ thở được và tim tôi đập rất nhanh. Nó thường  │                                              │                  │
│ biến mất sau 2 hoặc 3 phút nếu tôi có thể     │                                              │                  │
│ ngồi xuống và tập trung vào hơi thở. Gần đây  │                                              │                  │
│ tôi đã chụp MRI và tim tôi vẫn ổn......có ý   │                                              │                  │
│ kiến ​​gì không????                             │                                              │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ Tôi 36 tuổi, đang mang thai 12 tuần. Tôi có   │  Tôi đang mang thai, bạn có câu hỏi nào khác │ -0.162109375     │
│ một con gái 11 tuổi. Tôi có tiền sử bệnh PCod │ liên quan đến tình trạng này. Tôi muốn hỏi   │                  │
│ và lạc nội mạc tử cung. Tôi bị sảy thai trong │ thêm về tình trạng này. Tôi đang mang thai,  │                  │
│ lần mang thai đầu tiên và đó là sinh ba. cả   │ bạn có câu hỏi nào khác liên quan đến tình   │                  │
│ thai kỳ của tôi đều nhờ điều trị nhưng lần    │ trạng này. Tôi muốn hỏi thêm về tình trạng   │                  │
│ này nhờ ơn Chúa, tôi thụ thai tự nhiên. Tôi   │ này                                          │                  │
│ được tiêm fertygen hai lần một tuần, viên     │                                              │                  │
│ nang progestrone và bây giờ là viên nén nhưng │                                              │                  │
│ hôm qua tôi bị chảy máu với những cục máu     │                                              │                  │
│ đông lớn sau khi chụp chiếu, em bé vẫn ổn bên │     

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ query                                         ┃ model response                                 ┃ score          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ Xin chào, tôi đang gặp vấn đề với một khối u  │  Tôi đang lo lắng về tình trạng sức khỏe của   │ 0.1328125      │
│ sưng tấy ở một bên cổ gần cằm. đôi khi nó sẽ  │ mình và tôi muốn biết liệu tình trạng này có   │                │
│ rất đau và khối u sẽ “bùng nổ” đột ngột và    │ liên quan đến bệnh ung thư hay không. Tôi đang │                │
│ tôi có thể cảm thấy vị đắng đang trào lên cổ  │ tìm kiếm sự đánh giá và đánh giá tình trạng    │                │
│ họng mình. đó có phải là sưng hạch bạch huyết │ sức khỏe của mình. Tôi đang bị lo lắng về tình │                │
│ không?                                        │                                                │                │
├───────────────────────────────────────────────┼────────────────────────────────────────────────┼────────────────┤
│ Xin chào, không ngờ lại gặp được người thật,  │  Tôi rất lo lắng và mong muốn được tư vấn về   │ -0.130859375   │
│ nhưng vì bạn ở đây nên tôi sẽ kể cho bạn nghe │ tình trạng này. Tôi đang bị lo lắng về tình    │                │
│ về những triệu chứng mà tôi đang gặp phải.    │ trạng này và mong muốn được tư vấn về tình     │                │
│ Tôi nghĩ đó là những cơn hoảng loạn hoặc lo   │ trạng này. Tôi đang bị lo lắng về tình trạng   │                │
│ lắng, nhưng ai biết được. Tôi có cảm giác     │ này và mong muốn được tư vấn về tình trạng     │                │
│ căng cứng ở phía sau đầu, di chuyển lên thái  │                                                │                │
│ dương rồi xuống cổ họng và cuối cùng xuống    │                                                │                │
│ giữa ngực. Tôi rất xúc động khi điều này xảy  │                                                │                │
│ ra và sau tất cả những điều đó, tôi không thể │                                                │                │
│ thở được và tim tôi đập rất nhanh. Nó thường  │                                                │                │
│ biến mất sau 2 hoặc 3 phút nếu tôi có thể     │                                                │                │
│ ngồi xuống và tập trung vào hơi thở. Gần đây  │                                                │                │
│ tôi đã chụp MRI và tim tôi vẫn ổn......có ý   │                                                │                │
│ kiến ​​gì không????                             │                                                │                │
├───────────────────────────────────────────────┼────────────────────────────────────────────────┼────────────────┤
│ Tôi 36 tuổi, đang mang thai 12 tuần. Tôi có   │  Tôi đang mang thai, và tôi muốn biết liệu     │ -0.15625       │
│ một con gái 11 tuổi. Tôi có tiền sử bệnh PCod │ tình trạng này có liên quan đến tình trạng sức │                │
│ và lạc nội mạc tử cung. Tôi bị sảy thai trong │ khỏe của bạn hay tình trạng sức khỏe của em    │                │
│ lần mang thai đầu tiên và đó là sinh ba. cả   │ bé. Tôi đang mang thai, và tôi muốn biết liệu  │                │
│ thai kỳ của tôi đều nhờ điều trị nhưng lần    │ tình trạng này có liên quan đến tình trạng sức │                │
│ này nhờ ơn Chúa, tôi thụ thai tự nhiên. Tôi   │ khỏe                                           │                │
│ được tiêm fertygen hai lần một tuần, viên     │                                                │                │
│ nang progestrone và bây giờ là viên nén nhưng │                                                │                │
│ hôm qua tôi bị chảy máu với những cục máu     │                                                │                │
│ đông lớn sau khi chụp chiếu, em bé vẫn ổn bên │     

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ query                                         ┃ model response                                 ┃ score          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ Xin chào, tôi đang gặp vấn đề với một khối u  │  Tôi đang lo lắng về tình trạng sức khỏe của   │ 0.1357421875   │
│ sưng tấy ở một bên cổ gần cằm. đôi khi nó sẽ  │ mình và tôi muốn biết tình trạng này có liên   │                │
│ rất đau và khối u sẽ “bùng nổ” đột ngột và    │ quan đến bệnh ung thư hay không. Tôi đang tìm  │                │
│ tôi có thể cảm thấy vị đắng đang trào lên cổ  │ kiếm sự đánh giá và đánh giá tình trạng sức    │                │
│ họng mình. đó có phải là sưng hạch bạch huyết │ khỏe của mình. Tôi đang bị bệnh gì? Tôi đang   │                │
│ không?                                        │                                                │                │
├───────────────────────────────────────────────┼────────────────────────────────────────────────┼────────────────┤
│ Xin chào, không ngờ lại gặp được người thật,  │  Tôi rất lo lắng và mong được tư vấn về tình   │ -0.13671875    │
│ nhưng vì bạn ở đây nên tôi sẽ kể cho bạn nghe │ trạng này. Tôi đang bị lo lắng về tình trạng   │                │
│ về những triệu chứng mà tôi đang gặp phải.    │ này và mong được tư vấn về tình trạng này. Tôi │                │
│ Tôi nghĩ đó là những cơn hoảng loạn hoặc lo   │ đang bị lo lắng về tình trạng này và mong được │                │
│ lắng, nhưng ai biết được. Tôi có cảm giác     │ tư vấn về tình trạng này. Tôi                  │                │
│ căng cứng ở phía sau đầu, di chuyển lên thái  │                                                │                │
│ dương rồi xuống cổ họng và cuối cùng xuống    │                                                │                │
│ giữa ngực. Tôi rất xúc động khi điều này xảy  │                                                │                │
│ ra và sau tất cả những điều đó, tôi không thể │                                                │                │
│ thở được và tim tôi đập rất nhanh. Nó thường  │                                                │                │
│ biến mất sau 2 hoặc 3 phút nếu tôi có thể     │                                                │                │
│ ngồi xuống và tập trung vào hơi thở. Gần đây  │                                                │                │
│ tôi đã chụp MRI và tim tôi vẫn ổn......có ý   │                                                │                │
│ kiến ​​gì không????                             │                                                │                │
├───────────────────────────────────────────────┼────────────────────────────────────────────────┼────────────────┤
│ Tôi 36 tuổi, đang mang thai 12 tuần. Tôi có   │  Tôi đang mang thai và đang lo lắng về tình    │ -0.154296875   │
│ một con gái 11 tuổi. Tôi có tiền sử bệnh PCod │ trạng sức khỏe của em bé. Tôi muốn biết liệu   │                │
│ và lạc nội mạc tử cung. Tôi bị sảy thai trong │ tình trạng này có thể ảnh hưởng đến sự phát    │                │
│ lần mang thai đầu tiên và đó là sinh ba. cả   │ triển của em bé hay không. Tôi đang mang thai  │                │
│ thai kỳ của tôi đều nhờ điều trị nhưng lần    │ và đang lo lắng về tình trạng sức khỏe của em  │                │
│ này nhờ ơn Chúa, tôi thụ thai tự nhiên. Tôi   │                                                │                │
│ được tiêm fertygen hai lần một tuần, viên     │                                                │                │
│ nang progestrone và bây giờ là viên nén nhưng │                                                │                │
│ hôm qua tôi bị chảy máu với những cục máu     │                                                │                │
│ đông lớn sau khi chụp chiếu, em bé vẫn ổn bên │     

In [ ]:
ppo_trainer.model.policy.save_pretrained(configs['ppo_pretrain_model'])
torch.save(ppo_trainer.model.value_model.value_head.state_dict(), configs['ppo_pretrain_model']+'/value_head.pth')

tokenizer.save_pretrained(configs['ppo_pretrain_model'])